In [61]:
import os, yaml
import requests
from openai import OpenAI
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings.openai import OpenAIEmbeddings



In [62]:
#Setting the Credentials of OpenAI API
with open('credentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']

In [63]:
def complete_model(USER_MESSAGE):
    response = client.chat.completions.create(
                                            model = 'gpt-4o',
                                            messages = [
                                                        {"role": "system", "content" : "You are a helpful assitant to identify the tag for sense of a word"},
                                                        {"role": "user", "content": USER_MESSAGE}              
                                                        ],
                                            temperature=0,
                                            max_tokens=1500
                                            )
    return str(response.choices[0].message.content)

In [64]:
embedding_llm = HuggingFaceBgeEmbeddings(
                                        model_name = "BAAI/bge-small-en",
                                        model_kwargs = {'device': 'cuda'}, #CPU
                                        encode_kwargs = {'normalize_embeddings': False})

In [65]:
#Creating the tex Chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
                                        separator="\n",
                                        chunk_size=1000,
                                        chunk_overlap=200,
                                        )
    chunks = text_splitter.split_text(text)
    return chunks

In [66]:
#buidling the vector store
def get_vectorstore(text_chunks):
    vectorstore = FAISS.from_texts(
                                    texts=text_chunks, 
                                    embedding=embedding_llm
                                    )
    return vectorstore

In [67]:
def get_data():
    file= open("data.txt","r")
    txt=file.read()
    
    return txt

In [68]:
def data_pipeline():
    text = get_data()
    chunks = get_text_chunks(text)
    vectorstore = get_vectorstore(chunks)
    return vectorstore

In [69]:
vectorstore=data_pipeline()

In [38]:
#Not Required for the porgram. Displaying the relevant documents from Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k" : 3}, search_type="similarity")


In [70]:

docs = retriever.get_relevant_documents("Ringraziamento per il messaggio espresso dal Presidente riguardo l'emergenza COVID")
docs

[Document(page_content="Signor Presidente, onorevoli senatori, onorevole Ministro, la ringrazio per il messaggio di consapevolezza e impegno che oggi ha portato in quest'Aula e che già ieri sera dalla Camera è entrato nelle case degli italiani. Tutti noi abbiamo bisogno di questo messaggio. L'Italia ha necessità di questo messaggio, ma anche l'Europa e il mondo, e noi ci uniamo per trasmettere il senso di sicurezza che viene dalla convinzione che il nostro sistema sanitario e di protezione, composto da medici, infermieri, Protezione civile, Forze dell'ordine, Esercito e associazioni di volontariato è all'altezza di questa situazione per quanto questa sia in tanta parte inedita. Noi diciamo ancora grazie a tutti questi nostri concittadini, perché - permettetemi di sottolinearlo - ci danno il senso di quanto è contenuto all'articolo 4 della nostra Costituzione: «Ogni cittadino ha il dovere di svolgere, secondo le proprie possibilità e la propria scelta, un'attività o una funzione che con

In [72]:
#Finetuned UMberto model
from transformers import pipeline
classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-emotion',top_k=2)
prediction = classifier("Signor Presidente, onorevoli senatori, onorevole Ministro, la ringrazio per il messaggio di consapevolezza e impegno che oggi ha portato in quest'Aula e che già ieri sera dalla Camera è entrato nelle case degli italiani. Tutti noi abbiamo bisogno di questo messaggio.")
print(prediction)


[[{'label': 'joy', 'score': 0.9992088675498962}, {'label': 'sadness', 'score': 0.0005023473058827221}]]


In [73]:
#GPT 4o
prompt=f" Given the test, assign a emotional label from from [anger, disgust, happiness, sadness, fear, and surprise]. You need to refer to {docs} "

In [74]:
output=complete_model(prompt)

In [75]:
print(output)

The emotional label for the given text is "happiness." 

The speech conveys a sense of unity, gratitude, and confidence in the collective efforts of various sectors in combating the coronavirus. It emphasizes the importance of solidarity, the effectiveness of the healthcare system, and the positive outcomes of the measures being taken. The applause and congratulations at the end further reinforce the positive and uplifting tone of the message.
